In [ ]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import os
#os.chdir("..\\thermography_python\\")
os.chdir("..")
print(os.getcwd())

In [ ]:
import glob
import matplotlib.pyplot as plt
#from reg_tools import *

In [ ]:
from im_proc import *
from Segment import *
from registration import *
from read_data import *

In [ ]:
from retna import *
from retna.data_loader import *
from retna.view import *

In [ ]:
in_channel = 1
out_channel = 4
h_chans = [16,16,16,16,16]
model = Retna_V1(in_channel, out_channel, h_chans)


## Specify the data directory

## Run registration

In [ ]:
root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
fns = glob.glob(root + "*.H5")

In [ ]:
#root = "C:\\Users\\eac84\\Desktop\\IRT Surgery"
root = "T:/ProjectData/DIEP FLAP/"
xvi_fns = glob.glob( root + "*")
len(xvi_fns)

In [ ]:
for fn in fns:
    print(fn)    
    data = ThermoData(fn)
    output = register_datset(data)
    save_output(root,fn,output)      

In [ ]:
for fn in filenames:
    print(fn)    
    data = read_thermo(fn)
    output = reg_and_plot(data)
    save_output(root,fn,output_data)     

## Get Select Regions and Mask

In [ ]:
data = read_thermo(fn)
img = data[0]

In [ ]:
data = read_thermo(fn)
img = data[0]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10), sharex=True, sharey=True)
segments = get_regions(img, method=2)
ax[0].imshow(img)
ax[1].imshow(segments)

In [ ]:
import numpy as np 
data = np.array([100,200,300]).astype(np.uint16)
np.issubdtype(data.dtype,np.uint16)

In [ ]:
fn = filenames[3]
fn

In [ ]:
fn = filenames[2]
data = read_thermo(fn)

mask = plt.imread("../ML Training/Flap_label.jpg")[:,:,0]>100

frames = vis_3_panel(data, mask=mask)
render_frames("flap.mp4", frames )

## Measurement and Figure Making

In [ ]:
fn = xvi_fns[24]

In [ ]:
output = register_datset(im)

In [ ]:
plt.close("all")
## 11,18, 24

for n,fn in enumerate(fns):
    
    if "pre" not in fn: continue
    print(".", end="") 
    im,meta= read_H5(fn, t=5)
    #if len(im)<30: continue
    if im is None: continue
    print(n)
    plt.figure()
    plt.imshow(im)
    plt.title(n)

In [ ]:

im,_= read_H5(fns[24])


In [ ]:

im = im[5:-8]
x,y = 270,140
cmap = "gray"
draw_crosssection(im,x=x,y=y,t=3,cmap=cmap)

In [ ]:
plt.close("all")
classes = label_preop(im,tx=-20)
draw_labels(im, classes)
m = measure_preop(im,classes)

In [ ]:
all_classes = []
for t in range(10,len(im),5):

  classes = label_preop(im,tx=t)
  all_classes.append(classes)

In [ ]:
#root = "C:/Users/eac84/Desktop/Test_Flap_data/"
#fns = glob.glob(root +  "2*.H5")
im,meta= read_data.read_H5(fns[20])

In [ ]:
tx = 20
classes = label_preop(im,tx=tx)
draw = draw_labels_ontop(im, classes, tx=tx, a=0.2)
plt.figure()
plt.imshow(draw)

In [308]:
im2 = 1.* im - 30
im2 = im2 / 180.0
im2 = im2.clip(0,1)*255
im2 = im2.astype(np.uint8)

In [310]:


t0 = 5
tend = len(im)-20
tskip = 1

frames = []
for n,t in enumerate(range(t0,tend,tskip)):

  #if (t>40) & (t<45): continue
  print(".", end="")
  classes = label_preop(im,tx=t)
  draw = draw_labels_ontop(im2, classes, tx=t, a=0.2)
  frames.append(draw)


.........................................................................

In [309]:
plt.imshow(np.hstack((im2[0],im2[-30])))

In [ ]:
t = 50

classes = label_preop(im,tx=t)
draw = draw_labels_ontop(im, classes, tx=t, a=0.2)
plt.imshow(draw)

In [311]:
frames = np.array(frames)
frames2 = frames[:,40:400,20:560]
plt.imshow(frames2[0])

In [287]:
render_frames("./frames.mp4",frames2)

In [ ]:
def draw_labels_ontop(im, classes, tx=-1, ax=None, a=0.3):

    colors = [[0,0,0],[0,0,1],[0,1,0],[1,0,0],[0,1,1]]
    colors = np.array(colors)*255

    imx = im[(tx-3):(tx)].mean(axis=0)  
    imx = np.tile(imx[...,None],[1,1,3])
    
    #d_labels = labels/np.max(labels) * 255
    idx = classes[0]*0
    for n,cls in enumerate(classes):

        cls = bi_dilate(cls,1,1)
        if n==0: cls = cls & ~bi_erode(cls,5,0.5)
        if n==2: cls = bi_dilate(cls,2,1) & ~bi_erode(cls,1,0.5)

        idx[cls]=n+1

    L_rgb = colors[idx]

    draw = merge_layers(imx, L_rgb, a=0.5)
    L_rgb = draw_labels_annotations(classes)
    draw = merge_layers(imx, L_rgb, a=1)

    return draw


def draw_labels_annotations(classes):

    draw = classes[0]*0
    draw = np.tile(draw[...,None],[1,1,3])

    draw = draw_cross_navel(classes,draw)
    draw = draw_box_pref(classes, draw)

    return draw

def draw_cross_navel(classes,draw):

    bx = classes[1]
    if not np.any(bx): return draw
    x_c, y_c = np.argwhere(bx).sum(0)/bx.sum()
    x_c, y_c = int(x_c), int(y_c)

    draw[(x_c):(x_c+1),:] = [0,120,0]
    draw[:,(y_c):(y_c+1)] = [0,120,0]

    return draw

def draw_box_pref(classes, draw):

    bx = classes[2]
    bx = get_largest(bx)
    if not np.any(bx): return draw
    x, y = np.argwhere(bx).T
    x_min, x_max = int(x.min()), int(x.max())
    y_min, y_max = int(y.min()), int(y.max())

    draw[(x_min):(x_max),(y_min-1):(y_min+1)] = [255,0,0]
    draw[(x_min):(x_max),(y_max-1):(y_max+1)] = [255,0,0]

    draw[(x_min-1):(x_min+1),(y_min):(y_max)] = [255,0,0]
    draw[(x_max-1):(x_max+1),(y_min):(y_max)] = [255,0,0]

    return draw



def merge_layers(imx, L_rgb, a=0.3):

    bx = (L_rgb>0).any(axis=2)

    imx[bx] = (1-a)*imx[bx] + a*L_rgb[bx]

    draw = imx.astype(np.uint8)

    return draw




In [ ]:
bx = classes[1]
np.argwhere(bx).sum(0)/bx.sum()

In [ ]:
plt.imshow(draw_labels_annotations(classes))

In [ ]:
plt.imshow(classes[1])

In [ ]:
bx = classes[2]
bx = get_largest(bx)
x,y = np.argwhere(bx).T
x,y

In [ ]:
plt.imshow(bx)

In [ ]:
root = "../"
fns = glob.glob(root +  "2*.H5")

plt.close("all")
measurements = []
for n, fn in enumerate(fns):
    
    #with h5py.File(fn, 'r+') as fh:
    #    if "labels" in fh.keys():  continue
            
    ###############
    
    print(n, fn.split("\\")[-1], end ="\t \t ")      
    im,_= read_H5(fn)
    #if im.min()>100: continue
    ##############
    if len(im)<10: t0=1
    t0 = 10
    print(t0, end="\t")
    ###############
    im = im[t0:]
    
    ############################
    classes =None
    if 0:
        with h5py.File(fn, 'r+') as fh:
            if "labels" in fh.keys():  
                classes = fh["labels"][:]
            
        classes = np.squeeze(classes)
    ##################################
    try:
        if classes is None:     classes = label_preop(im,tx=-5)
        classes = classes[None,:]
        
        with h5py.File(fn, 'r+') as fh:
            if "labels" in fh.keys():  del fh["labels"]
            fh.create_dataset("labels", data=classes, compression="lzf" )        
    except:
        print("Error with", fn)
        
    print("")

    
    #m = measure_preop(im,classes)
    #measurements.append(m)

In [ ]:
im,_= read_H5(fns[0])

In [ ]:
im = im/(2**16)*(2**8)

In [ ]:
    fig,axs = plt.subplots(2,2)
    plt.subplots_adjust(left=0.02, bottom=0.02, 
            right=0.98, top=0.98, wspace = 0.05, hspace=0.05)
    axs = axs.ravel()
    [ax.grid(True) for ax in axs]
    #axs = [plt.subplot2grid((1,2), (0,0)), axs[0,1],axs[1,1]]
    
    axs[0].imshow(im.mean(axis=0),cmap="gray")
    draw_labels(im, classes, axs[1])
    
    ax = axs[2]
    ax.plot(im.min(axis=(1,2)))
    ax.plot(im.mean(axis=(1,2)))
    ax.plot(np.diff(1.*im,axis=0).mean(axis=(1,2)))
    ax.plot([t0,t0],[10,200])

In [ ]:
classes = label_preop(im, tx=20)
draw_labels(im, classes, tx=10)

In [ ]:
Loss = []

tx = -20
Lx = label_preop(im, tx=tx)

T = range(5,len(im)+tx,4)
X = np.array(T) / 15

for tx in T:
    
    L_ = label_preop(im, tx=tx)
    DL = 1-(dice_loss(L_, Lx, smooth = .0001)[0])
    Loss.append(DL)
    
Loss = np.array(Loss)

plt.close("all")
plt.plot(X, Loss, "-o")
plt.title("Dice Score of Segmentation over time")
plt.xlabel("Minutes")
plt.grid(True)
plt.xlim(plt.xlim())
plt.plot(plt.xlim(),[0,0],"k")
plt.plot(plt.xlim(),[1,1],"k")
plt.legend(["Abdomen", "Umbilicus", "Preforators","Artifacts"])

In [ ]:
M = []
S = []

tx = -15
Lx = label_preop(im, tx=tx)

T = range(0,len(im)+tx,2)

for tx in T:
    
    #Lx = label_preop(im, tx=tx)

    means = [im[tx][m].mean() for m in Lx]
    devs = [im[tx][m].std() for m in Lx]
    M.append(means)
    S.append(devs)
    

In [ ]:
Y = np.array(M) / 4.
E = np.array(S) / 4. / 2.

N = np.array([L.sum() for L in Lx])

X = np.array(T) / 15

Yp = Y + E
Ym = Y - E

plt.close("all")
plt.plot(X,Y, "-o")
plt.title("Heating Profile by Region")
plt.xlabel("Minutes")
plt.ylabel("Temperature (Celcius)")
plt.grid(True)
plt.xlim(plt.xlim())

#for n in range(Yp.shape[1]): plt.fill_between(X, Yp[:,n], Ym[:,n], alpha=0.1)

plt.legend(["Abdomen", "Umbilicus", "Preforators","Artifacts"])

In [ ]:
Ls_T = []
tx = -10
Lx = label_preop(im, tx=tx)
for tx in np.linspace(10,len(im)+tx,10).astype(int):

    print(tx)
    L_ = label_preop(im, tx=tx)
    DL = 1-(dice_loss(L_, Lx, smooth = .001)[0])
    L_ = (L_ * np.array([1,2,3,4])[None,:,None,None]).mean(axis=1)[0]
    draw = np.vstack([im[tx]/255, L_])

    Ls_T.append(draw)

#Ls_T = np.array(Ls_T) 
plt.imshow(np.hstack(Ls_T),cmap="jet")

In [ ]:
np.linspace(5,len(im)-tx,5).astype(int)

In [ ]:
    L_ = label_preop(im, tx=tx)
    L_ = (L_ * np.array([1,2,3,4])[None,:,None,None]).mean(axis=1)

In [ ]:
L_ = label_preop(im, tx=-1)
dL = label2rgb(L_[2])
L2 = label_nearest(L_[2], radius=np.inf, blockSeeds = True)
dL2 = label2rgb(L2)
dL[L_[0] & ~L_[2]] = .5*dL2[L_[0] & ~L_[2]]
dL[L_[1]] = 0 
#dL[L_[0] & ~L_[2]] = 100
plt.close("all")
plt.imshow(dL)

In [ ]:
im,_= read_data.read_H5(filenames[5])
im = im[1:,:450,:550]

In [ ]:
plt.imshow(im[-3]*1.)

In [ ]:
render_frames("test.mp4", im_fr , fps=10)

In [ ]:
import scipy
def timesample(im):
    fig,axs = plt.subplots(1,3)
    plt.subplots_adjust(left=0.02, bottom=0.04, 
            right=0.98, top=0.9, wspace = 0.05, hspace=0)
    [ax.grid(True) for ax in axs]
    #axs = [plt.subplot2grid((1,2), (0,0)), axs[0,1],axs[1,1]]
    t = -5

    axs[0].imshow(im[10],cmap="gray")
    #x1,x2,y1,y2 = 120,600,150,450
    x1,x2,y1,y2 = 80,550,80,350
    xs,ys = 80,60
    xrange = np.arange(x1,x2,xs)
    yrange = np.arange(y1,y2,ys)

    xrange = np.linspace(200,400,5).astype(int)
    yrange = np.linspace(200,350,5).astype(int)

    for x in xrange:
        for y in yrange:

            I = im[t,y,x]
            clr = np.array([I,0,255-I])/255
            axs[0].plot(x,y,"o", color = clr, ms=10)
            sig = im[:,y,x]
            axs[1].plot(sig, color= clr )
            sig = scipy.signal.medfilt(sig,3) 
            axs[2].plot(sig, color= clr )
            
    axs[1].set_ylim(0,255)
    axs[2].set_ylim(0,255)

    

In [ ]:
def draw_labels(im, classes, ax=None):
    dPred = 1.*im[-3] - im[0]
    bFlap,bMiss,bNav,bPref = classes 
    labels = np.sum([bFlap,2*bMiss,3*bNav,4*bPref],axis=0)
    d_labels = labels/np.max(labels) * 255
    draw1 = np.hstack((im[0],im[-3],))
    draw2 = np.hstack((dPred+125,d_labels)) 
    draw = np.vstack((draw1,draw2))

    ax.imshow(draw, cmap="jet")
    ax.axis("off")  

In [ ]:
## Annotate using NAPARI

In [ ]:
import napari
fn = fns[17]
im_data,_ = read_data.read_H5(fn)
im_names = ["thermography"]
maps = ["gray"]
viewer = napari.view_image(im_data) 

v = napari.current_viewer()

In [ ]:
with h5py.File(fn, 'r+') as fh:
    if "labels" in fh.keys():  
        labels = fh["labels"][:]

In [ ]:
labels = np.dot(labels.transpose(0,2,3,1),[1,2,3,4])

In [ ]:
label_names = ["flap","1","2","3"]
c = 0
v.add_labels(labels[None,:], name=label_names[c])

In [ ]:
for c in range():
    v.add_labels(labels[None,:], name=label_names[c])
    #v.add_labels(labels[:,c], name=label_names[c])

In [ ]:
v = napari.current_viewer()

In [ ]:
np.tile(labels,[len(im_data),1,1,1]).shape

In [ ]:
root = "C:/Users/eac84/Desktop/Test_Flap_data/"
fns = glob.glob(root +  "2*.H5")

In [ ]:
napari.run()

In [ ]:
labels.shape

In [ ]:
def dice_loss(pred, target, smooth = .0001):

    if pred.ndim == 3: pred = pred[None,:]
    if target.ndim == 3: target = target[None,:]

    if pred.shape[1] != target.shape[1]:
        print("loss warning - Shapes are different sizes",
                pred.shape[1],target.shape[1])

    intersection = 2*(pred * target).mean(axis=(2,3))
    combination =  (pred**2 + target**2).mean(axis=(2,3))
    dsc = (intersection + smooth) / (combination+smooth) 
    dsc = (1 - dsc)
    return dsc